# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install cartopy geoviews pyproj


# Import API key
from api_keys import geoapify_key

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 20.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.2/563.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.0
    Uninstalling bokeh-3.6.0:
      Successfully uninstalled bokeh-3.6.0


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points(
    'Lng', 'Lat',                # Longitude and Latitude columns for location
    geo=True,                    # Enables geographic plotting
    size='Humidity',             # Sets point size based on humidity
    color='Humidity',            # Colors points based on humidity
    colormap='coolwarm',         # Color scheme for humidity
    tiles='OSM',                 # OpenStreetMap background
    title='City Humidity Levels',
    frame_width=800, 
    frame_height=500
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) &   # Example temperature range
    (city_data_df['Humidity'] >= 30) & (city_data_df['Humidity'] <= 60)     # Example humidity range
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
34,34,naberera,-4.2000,36.9333,21.12,54,94,6.38,TZ,1666108252
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
print(hotel_df.columns)

# Display sample data
hotel_df.head()

Index(['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'], dtype='object')


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
34,naberera,TZ,-4.2000,36.9333,54,
63,banda,IN,25.4833,80.3333,52,
86,rafaela,AR,-31.2503,-61.4867,42,
106,palencia,ES,42.4167,-4.5000,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key,
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
naberera - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
rafaela - nearest hotel: No hotel found
palencia - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
cabra - nearest hotel: No hotel found
sao teotonio - nearest hotel: No hotel found
san cristobal - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
prieska - nearest hotel: No hotel found
jalu - nearest hotel: No hotel found
kingman - nearest hotel: No hotel found
tandil - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
viseu - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
narwar - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
kalemie - nearest hotel: No hotel found
barroso - nearest hotel: No hotel found
vryburg - nearest hotel: No hotel found
lahij - nearest hotel: No hotel found
chivilco

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
34,naberera,TZ,-4.2000,36.9333,54,No hotel found
63,banda,IN,25.4833,80.3333,52,No hotel found
86,rafaela,AR,-31.2503,-61.4867,42,No hotel found
106,palencia,ES,42.4167,-4.5000,30,No hotel found
152,kahului,US,20.8947,-156.4700,60,No hotel found
170,sur,OM,22.5667,59.5289,40,No hotel found
196,cabra,ES,37.4725,-4.4421,30,No hotel found
218,sao teotonio,PT,37.5128,-8.7071,60,No hotel found
229,san cristobal,VE,7.7669,-72.2250,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
city_data_df = city_data_df.merge(hotel_df[['City', 'Hotel Name']], on='City', how='left')

hotel_df['Hover Info'] = hotel_df.apply(lambda row: f"{row['City']}, {row['Country']} - Hotel: {row['Hotel Name']}", axis=1)

# Display the map
city_data_df.hvplot.points(
    'Lng', 'Lat',  # Longitude and Latitude columns for location
    geo=True,  # Enables geographic plotting
    size='Humidity',  # Sets point size based on humidity
    color='Humidity',  # Colors points based on humidity
    colormap='coolwarm',  # Color scheme for humidity
    tiles='OSM',  # OpenStreetMap background
    hover_cols=['Hover Info'],  # Adds hover information
    frame_width=800,
    frame_height=500
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)